In [ ]:
import pandas as pd
import numpy as np
data = pd.read_csv('polarity_labeling.csv')
data.head()

In [ ]:
def Make_matrix(df):
    # unique()는 특정 시리즈 데이터에서 유일한 값들을 넘파이배열로 출력
    product_list = df['product'].unique()
    matrix = np.zeros((len(product_list), len(product_list)), dtype=np.float32)
    
    # 불필요한 데이터 제거
    df = df[df['polarity'] != 0]
    for i, p in enumerate(product_list):
        df_re = df[df['product'] == p]
        df_cs = df_re[df_re['CS_SS'] == 'CS']
        df_ss = df_re[df_re['CS_SS'] == 'SS']
        
        if i%10 == 0:
            print(i)
        
        for j, t in enumerate(product_list):
            if i == j:
                # ss에 의한 웨이트 계산
                ss_data = df_ss['polarity'].value_counts()
                if len(ss_data.index) == 2:
                    matrix[i,j] = int(ss_data[ss_data.index == 1]) / int(ss_data[ss_data.index == -1])
                elif 1 not in ss_data.index:
                    matrix[i,j] = 0
                else:
                    matrix[i,j] = int(ss_data[ss_data.index == 1])

            else:
                # cs에 의한 웨이트 계산
                cs_data = df_cs[df_cs['target'].str.contains(t) == True]['polarity'].value_counts()
                if len(cs_data.index) == 2:
                    matrix[i,j] = int(cs_data[cs_data.index == 1]) / int(cs_data[cs_data.index == -1])
                elif 1 not in cs_data.index:
                    matrix[i,j] = 0
                else:
                    matrix[i,j] = int(cs_data[cs_data.index == 1])
    
    # 매트릭스를 데이터프레임형식으로 바꿈
    dff = pd.DataFrame(matrix, index = product_list, columns = product_list)
    dff.loc['sum'] = dff.sum()

    return dff

In [ ]:
df = Make_matrix(data)

In [ ]:
df.to_excel('matrix_data.xlsx')

In [ ]:
def Make_Rank(df, d = 0.85, MAX_ITERATIONS = 50, threshold = 0.0001):
    # sum = 0 인 열 전체를 지우는 함수
    df = df.T
    df=df[df['sum'] != 0].T
    
    product_list = df.columns
    
    # 초기 스코어 설정( 1 )
    score = np.ones(len(product_list))
    
    # 최대 반복 횟수 50 설정
    for iter in range(0,MAX_ITERATIONS):
        print(iter)
        prev_score = np.copy(score)
        
        for i, p in enumerate(product_list):
            # 넘파이 배열의 사칙연산을 사용하여 for문을 줄임
            score[i] = (1-d) + d*sum(df.loc[p] / df.loc['sum'] * prev_score)

        # 반복 중 수치 변화가 일정 값 미만으로 떨어지면 코드 종료
        if np.sum(np.fabs(prev_score-score)) <= threshold:
            print('요기서 끝 :', iter)
            break
    
    dff = pd.DataFrame(score, index = product_list)
    
    # 스코어 정규화
    sum_num = float(dff.sum())
    dff = dff.apply(lambda x: x/sum_num)
    
    # 내림차순으로 랭킹 배열
    dff = dff.sort_values(0,ascending=[False])
    
    return dff

In [ ]:
dff = Make_Rank(df)
dff.head(10)